# Interactive fitting with Minuit

The following script is an illustration of fitting, where it is possible to alter the fitting parameters interactively.

Adapted from iMinuit documentation and examples: https://scikit-hep.org/iminuit/

### Authors: 
- Malthe Nordentoft (Niels Bohr Institute)
- Troels C. Petersen (Niels Bohr Institute)

### Date:    
- 09-11-2024 (latest update)

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
from iminuit import Minuit, cost

In [2]:
# Generate the data, which consists of two overlapping Gaussians:
data = np.hstack([np.random.randn(1000)*1.1, np.random.randn(1000)*0.6+3.1])

# Bin the data in order to draw it:
Nbins = 100
xmin, xmax = -4.0, 6.0
binwidth = (xmax-xmin) / Nbins
freq, bins = np.histogram(data, bins = np.linspace(xmin, xmax, Nbins))
bins = (bins[1:] + bins[:-1])/2
freq_err = np.sqrt(freq)

# For ChiSquare fitting:
x = bins[freq > 0]
y = freq[freq>0]
sy = freq_err[freq >0]
print("Number of non-empty bins:", len(x))

Number of non-empty bins: 82


In [3]:
# Fitting functions:
def double_gaussian(x, N, mu1, mu2, sigma1, sigma2, f):
    return N * binwidth * (f*stats.norm.pdf(x, mu1, sigma1) + (1-f)*stats.norm.pdf(x, mu2, sigma2))

In [4]:
# ChiSquare fit (of course binned):
c = cost.LeastSquares(x, y, sy, double_gaussian)

# Unbinned likelihood fit:
#c = cost.UnbinnedNLL(data, double_gaussian)
#binwidth = 1.0

# Set the initial values (very important) and the allowed parameter ranges (for widget):
m = Minuit(c, N = 1000, mu1 = 0, mu2 = 4, sigma1 = 1.0, sigma2 = 0.5, f = 0.5)
m.limits["N"] = (0, 5000)
m.limits["mu1", "mu2"] = (-10, 10)
m.limits["sigma1", "sigma2"] = (0.01, 3)
m.limits["f"] = (0, 1)

In [5]:
m.interactive()